# Pentomino pipeline validation

このノートブックでは、画像処理パイプラインの各ステージを段階的に実行し、途中結果を可視化するための手順をまとめています。

## 使い方
1. `assets/puzzles/` フォルダに解析したい盤面画像を配置します。
2. 下記の `IMAGE_PATH` 変数をそのファイルに合わせて更新します。
3. 以降のセルを上から順に実行し、各ステージの出力を確認します。

In [ ]:
from pathlib import Path
import logging

import numpy as np

from pento import preprocessing, segmentation, classification, normalization
from pento.pipeline import PentominoArchiver
from pento.storage import JsonSolutionArchive
from pento.visualization import (
    format_label_grid,
    grid_to_labeled_pieces,
    labeled_pieces_to_grid,
    plot_grid_cells,
    plot_label_grid,
    show_board_image,
)

logging.basicConfig(level=logging.INFO)

In [ ]:
IMAGE_PATH = Path('assets/puzzles/sample.jpg')  # <- 解析したい画像に変更
ARCHIVE_PATH = Path('notebooks/solutions.jsonl')

IMAGE_PATH, ARCHIVE_PATH

In [ ]:
# 1. 画像読み込みと盤面抽出
original = preprocessing.load_image(IMAGE_PATH)
board = preprocessing.extract_board_region(original)

show_board_image(board, title='Extracted board view')

In [ ]:
# 2. グリッド分割
grid_cells = segmentation.segment_grid(board)
grid_cells.shape

In [ ]:
# (任意) 分割結果をまとめて可視化
fig, _ = plot_grid_cells(grid_cells)
fig

### 3. ピース識別
デモ用として、このノートブックでは 6×10 のサンプルラベルグリッド `SYNTHETIC_LABEL_GRID` を定義しています。実際の盤面に合わせたラベルが得られたら、この配列を差し替えてください。分類器が実装済みであれば `classification.label_pieces(grid_cells)` を直接呼び出して自動推論を行えます。


In [ ]:
SYNTHETIC_LABEL_GRID = np.array([
    list("FFFFFIIIII"),
    list("LLLLLPPPPP"),
    list("NNNNNTTTTT"),
    list("UUUUUVVVVV"),
    list("WWWWWXXXXX"),
    list("YYYYYZZZZZ"),
], dtype="<U1")

label_grid = SYNTHETIC_LABEL_GRID.copy()
print(format_label_grid(label_grid))
plot_label_grid(label_grid)

pieces = classification.label_pieces(label_grid)
[piece.name for piece in pieces]


### 4. 正規化とアーカイブ
分類結果 `pieces` を正規化処理に渡すと、共通フォーマットの盤面表現を得られます。生成されたグリッドはアーカイブ保存や比較に利用できます。


In [ ]:
label_grid_from_pieces = labeled_pieces_to_grid(pieces)
print(format_label_grid(label_grid_from_pieces))
plot_label_grid(label_grid_from_pieces)

canonical = normalization.to_canonical_solution(pieces)
canonical.grid


## パイプライン全体をまとめて動かす
分類器が実装済みであれば、`PentominoArchiver` を使って一括実行できます。
ログ出力で各ステージの進行状況を確認してください。

In [ ]:
# archiver = PentominoArchiver(archive=JsonSolutionArchive(ARCHIVE_PATH))
# result = archiver.process(IMAGE_PATH)
# display(result.canonical_solution.grid)
# plot_label_grid(labeled_pieces_to_grid(result.labeled_pieces))